# Discrete gradient operators

This notebook collects my progress towards the definition of the operator

$$\nabla_h:W_h \rightarrow \Theta_h$$

as defined in (Bartels, 2015) §8.2.1.

## First goal: derivative of a piecewise linear function

** To do for this goal:** A rewrite in C++?

Let $u \in V$, where $V$ is a CG1 finite element space. Then $u'$ is piecewise constant (i.e. it is in DG0). We want to compute the matrix $D_h$ for 

$$\nabla_h:V \rightarrow V'$$

such that $(D_h u)_j$ = $(\nabla_h u)_j = u_j'$. Note that we are using the same  notation for a function $u$ and for the vector of its coefficients in the basis of the space it lives on.

Since $u'$ is not in the same space as $u$, in FEniCS we need to `project()` it onto $V'$, which is costly. What we are doing is to manually compute the derivative. This should be easy since

$$ u = \sum_i \alpha_i \phi_i $$

where $V = \text{span} \{\phi_i\} $, hence:

$$ u' = \sum_i \alpha_i \phi_i'. $$

We can use `evaluate_basis_derivatives()` for this purpose, assembling the transformation matrix by hand.

We define $V' = \text{span} \{\phi_i'\}$, a DG0 space, and compute the linear mapping of dofs from $V$ to $V'$, then apply this mapping to the coefficients of $u$ and construct $u' \in V'$ with these coefficients.

For simplicity, consider first a 1D mesh. Let $i \in \{0,1,...,M\}$ be an index over the cells of the mesh such that:

* $u_i,u_{i+1}$ are the coefficients for the dofs in $V$ over cell $i$
* $u_i'$ is the coefficient for the only dof in $V'$ (the constant 1) over cell $i$

Then we only need to compute $\phi_i'$ for each $i$ to obtain the new vector of coefficients $(u_i')_{i=0}^{N_{V'}}$:

$$
\left(\begin{array}{ccccc}
  \phi_0' & \phi_1' &  &  & \\
  & \phi_1' & \phi_2' &  & \\
  &  & \phi_2' & \phi_3' & \\
  &  &  & \phi_3' & \phi_4'
\end{array}\right)  \left(\begin{array}{c}
  u_0\\
  u_1\\
  u_2\\
  u_3\\
  u_4
\end{array}\right) = \left(\begin{array}{c}
  u_0'\\
  u_1'\\
  u_2'\\
  u_3'
\end{array}\right).$$

Note that in this particular instance and with the assumptions made, we obtain a band matrix and computing $u'$ is actually a matter of computing the dot product of a "subset" of the vector of constants $(\phi_0', ..., \phi_{N_{V'}}')$ with $u$.

However in other situations, e.g. in higher dimensions, the ordering of the nodes won't be as convenient. We proceed then more generally as follows: Let $i$ run over all cell indices and let $\iota_i$ be the local-to-global mappings for $V$ and $\iota_i'$ for $V'$. Initialise the matrix $M$ of the discrete gradient to zero. At each step of the construction of $M$ we edit row $r = \iota_i'(0)$, i.e. the row whose product by $(u)_j$ will return the coefficient for the dof in $V'$ corresponding to cell $i$.

Now, for every vertex $v_j,\ j \in \{0,...,d\}$ in cell $i$ compute the value of the derivative of the global dof $\phi_{\iota_r(j)}$ and set

$$M_{r j} = \phi_{\iota_r(j)},\ j \in \{0,...,d\}.$$

In [ ]:
from itertools import chain
from dolfin import *
from petsc4py import PETSc
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

np.set_printoptions(precision=2)

def __nbinit__():
    global __all__
    __all__ = ['compute_discrete_gradient_dense', 
               'compute_discrete_gradient',
               'compute_dkt_gradient']
    global parameters
    # instruct FFC to produce the code for evaluate_basis_derivatives()
    parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

__nbinit__()

In [ ]:
def discrete_gradient_operator_dense(V, Vp):
    """ Build dense matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type CG1.
        Vp: Target (Vector)FunctionSpace of type DG0.
        
    Returns
    -------
        Dh: np.ndarray of shape (Vp.dim(), V.dim()) such that,
            for u in V:
        
                np.dot(Dh, u.vector().array())
              
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'CG1', "Need a CG1 source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x DG0' % gdim or\
           Vp.ufl_element().shortstr()[:3] == 'DG0',\
           "I need a DG0 (Vector)FunctionSpace of the right number of components"
    assert Vp.mesh().num_cells() * gdim == Vp.dim(),\
           "FunctionSpace dimensions don't match."
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Meshes don't match"

    dm = V.dofmap()
    dmp = Vp.dofmap()
    e = V.element()
    coords = V.tabulate_dof_coordinates().reshape((-1, gdim))
    coordsp = Vp.tabulate_dof_coordinates().reshape((-1, gdim))

    Dh = np.zeros((Vp.dim(), V.dim()))
    warning("Assuming all cells have the same number of dofs")
    vals = np.zeros(dm.num_element_dofs(0)*gdim)
    for cell_id in range(Vp.mesh().num_cells()):
        rows = dmp.cell_dofs(cell_id)
        columns = dm.cell_dofs(cell_id)
        dof_coords = coords[columns,:]
        point = dof_coords[0] # Any point in the (closure of the) cell will do.
        e.evaluate_basis_derivatives_all(1, vals, point, dof_coords, 1)
        #print("rows=%s, columns=%s, point=%s, dof_coords=%s, vals=%s" %
        #     (rows, columns, point, dof_coords, vals.round(2)))
        # Reshape [df_1/dx, df_1/dy, df_2/dx, df_2/dy, ...] into
        # [[df_1/dx, df_2/dx, ...],[df_1/dy, df_2/dy, ...]] :
        #Dh[FIXTHIS:"rows, columns"] = vals.reshape((-1, gdim)).T.copy()
        for offset in range(gdim):
            Dh[rows[offset], columns] = vals[offset::gdim].copy()

    return Dh

def compute_discrete_gradient_dense(V, Vp, u):
    """ Returns the discrete gradient of u.
    NOTE: this implementation uses dense matrices internally.

    Arguments
    ---------
        V: Source FunctionSpace of type CG1. (redundant...)
        Vp: Target (Vector)FunctionSpace of type DG0.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """

    Dh = discrete_gradient_operator_dense(V, Vp)
    up = Function(Vp)
    up.vector().set_local(np.dot(Dh, u.vector().array()))
    up.vector().apply('insert')

    return up

In order to test the previous code we start with an expression whose derivative we can compute:

In [ ]:
# instruct FFC to produce the code for evaluate_basis_derivatives()
parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

LEFT, RIGHT = -pi, 2*pi
mesh = IntervalMesh(100, LEFT, RIGHT)
V = FunctionSpace(mesh, "CG", 1)
Vp = FunctionSpace(mesh, "DG", 0)

#x = SpatialCoordinate(mesh)
#f = sin(x[0])
#u = project(f, V)
u = interpolate(Expression("sin(x[0])", element=V.ufl_element()), V)
dg_u = compute_discrete_gradient_dense(V, Vp, u)

# Test
#up = project(f.dx(0), Vp)
up = interpolate(Expression("cos(x[0])", element=Vp.ufl_element()), Vp)
print("Error: %e" % (dg_u.vector() - up.vector()).norm('linf'))
plot(up)
plot(dg_u)
_ = pl.xlim((LEFT*1.1, RIGHT*1.1)), pl.ylim((-1.1,1.1))

## A detour using projections

The following method computes the solution via a projection onto DG0 but instead of letting FEniCS assemble the mass matrix, we make use of the fact that it is diagonal with entries given by the cell volume. Then it is trivial to invert it and multiply the rhs of the projection problem. The idea and code are taken from [this question on FEniCS Q&A](https://fenicsproject.org/qa/12634/gradient-of-a-cg-order-one-element?show=12646#a12646).

In [ ]:
from dolfin import *
import numpy as np

mesh = UnitCubeMesh(30, 30, 30)
W = FunctionSpace(mesh, 'CG', 1)
Q = VectorFunctionSpace(mesh, 'DG', 0)
p, q = TrialFunction(Q), TestFunction(Q)

f = Function(W)
f_vec = f.vector()
f_vec.set_local(np.random.rand(f_vec.local_size()))
f_vec.apply('insert')

We first compute the gradient via a projection:

In [ ]:
%%time

L = inner(grad(f), q)*dx
M = assemble(inner(p, q)*dx)
b = assemble(L)

grad_f0 = Function(Q)
x0 = grad_f0.vector()
_ = solve(M, x0, b)

MiroK's idea (which amounts to building our matrix $D_h$) in the forum is to "assemble the action of Minv onto the rhs - no mass matrix needed". The improvement in speed is huge:

In [ ]:
%%time

MinvL = (1/CellVolume(Q.mesh()))*inner(grad(f), q)*dx
x = assemble(MinvL)

In [ ]:
grad_f = Function(Q, x)
print("Error: %e" % (x - x0).norm('linf'))

Note that in the first approach we compute $M$, then solve $M x = b$ and in the second we compute $M^{-1}$ directly to obtain $x = M^{-1} b$.

### How does the assembly of the projection problem work?

Write the compiled form to a file and check `dgrad_cell_integral_0_otherwise::tabulate_tensor()` for the implementation. Recall that `u.dx()` in the form is the derivative of a terminal node in UFL so it is the responsibility of FFC to fill that. How does this work again?

In [ ]:
import ffc
with open("/tmp/dgrad.h", "wt") as fd:
    out = ffc.compile_form(MinvL, prefix="dgrad")
    fd.write(out[0])

## And back

We can use our function instead. This will be much slower and also use a lot of memory because of the dense matrix we are using. The error is different, though. **Could this be significant?**

In [ ]:
%%time

dg_f = compute_discrete_gradient_dense(W, Q, f)

In [ ]:
print("Error: %e" % (dg_f.vector() - grad_f0.vector()).norm('linf'))

## Use the PETSc backend

We achieve next a (back-of-the-napkin) speed improvement of up to 50 times wrt. the implementation with dense matrices and stay in the same range of running times than the method using `project()`. We can only test for manageable matrix sizes but we can now of course manage bigger matrices. The speedup depends greatly on the size of the matrices.

In [ ]:
def discrete_gradient_operator(V, Vp):
    """ Build sparse PETSc matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type CG1.
        Vp: Target (Vector)FunctionSpace of type DG0.
        
    Returns
    -------
        Dh: PETSc sparse matrix such that for u in V,
              Dh * u.vector().array()
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """   
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'CG1',\
           "Need a CG1 source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x DG0' % gdim or\
           Vp.ufl_element().shortstr()[:3] == 'DG0',\
           "I need a DG0 (Vector)FunctionSpace of the right number of components"
    assert Vp.mesh().num_cells() * gdim == Vp.dim(),\
           "FunctionSpace dimensions don't match"
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Number of cells in the meshes don't match"

    dm = V.dofmap()
    dmp = Vp.dofmap()
    e = V.element()
    coords = V.tabulate_dof_coordinates().reshape((-1, gdim))
    coordsp = Vp.tabulate_dof_coordinates().reshape((-1, gdim))

    Dh = PETSc.Mat()
    Dh.create(PETSc.COMM_WORLD)  # What is this?
    Dh.setSizes([Vp.dim(), V.dim()])
    Dh.setType("aij")
    Dh.setUp()
    warning("Assuming all cells have the same number of dofs")
    vals = np.zeros(dm.num_element_dofs(0)*gdim)
    # TODO: check this for parallel operation...
    istart, iend = Dh.getOwnershipRange()
    print ("This process owns the range %d - %d" % (istart, iend))
    for cell_id in range(Vp.mesh().num_cells()):
        rows = dmp.cell_dofs(cell_id)
        rows = rows[(rows>=istart) & (rows < iend)]
        columns = dm.cell_dofs(cell_id)
        dof_coords = coords[columns,:]
        point = dof_coords[0] # Any point in the (closure of the) cell will do.
        e.evaluate_basis_derivatives_all(1, vals, point, dof_coords, 1)
        #print("rows=%s, columns=%s, point=%s, dof_coords=%s, vals=%s" %
        #     (rows, columns, point, dof_coords, vals.round(2)))
        # Reshape [df_1/dx, df_1/dy, df_2/dx, df_2/dy, ...] into
        # [[df_1/dx, df_2/dx, ...],[df_1/dy, df_2/dy, ...]] :
        # NOTE: the copy() is necessary!
        Dh.setValues(rows, columns, vals.reshape((-1, gdim)).T.copy())
    Dh.assemble()
    return Dh

def compute_discrete_gradient(V, Vp, u):
    """ Returns the discrete gradient of u.

    Arguments
    ---------
        V: Source FunctionSpace of type CG1. (redundant...)
        Vp: Target (Vector)FunctionSpace of type DG0.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """
    Dh = discrete_gradient_operator(V, Vp)
    du = Function(Vp)
    petsc_u = as_backend_type(u.vector()).vec()
    petsc_du = as_backend_type(du.vector()).vec()
    
    Dh.mult(petsc_u, petsc_du)
    du.vector().apply('insert')

    return du

In [ ]:
# instruct FFC to produce the code for evaluate_basis_derivatives()
parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

LEFT, RIGHT = -pi, 2*pi
mesh = IntervalMesh(1000, LEFT, RIGHT)
V = FunctionSpace(mesh, "CG", 1)
Vp = FunctionSpace(mesh, "DG", 0)

#Dh = discrete_gradient_operator(V, Vp)
u = interpolate(Expression("sin(x[0])", degree=1), V)
du0 = interpolate(Expression("cos(x[0])", degree=0), Vp)

In [ ]:
%%time
du = compute_discrete_gradient(V, Vp, u)

In [ ]:
print("Error: %e" % (du.vector() - du0.vector()).norm('linf'))
_ = plot(du)

This is faster than the solution with dense matrices but obviously still much slower (a little below 40 times) than building $M^{-1}$ directly because of the hideously slow python loops:

In [ ]:
%%time

dgrad_f = compute_discrete_gradient(W, Q, f)

In [ ]:
print("Error: %e" % (dgrad_f.vector() - grad_f0.vector()).norm('linf'))

## DKTs

Theory, stuff...

In [ ]:
def dkt_gradient_operator(V, Vp):
    """ Build sparse PETSc matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type DKT.
        Vp: Target (Vector)FunctionSpace of type P2
            (WITHOUT a hierarchical basis)
        
    Returns
    -------
        Dh: PETSc sparse matrix such that for u in V,
              Dh * u.vector().array()
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """   
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'DKT',\
           "I need a DKT source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x CG2' % gdim,\
           "I need a CG2 VectorFunctionSpace with %d components" % gdim
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Number of cells in the meshes don't match"
    dm = V.dofmap()
    dmp1, dmp2 = Vp.sub(0).dofmap(), Vp.sub(1).dofmap()
    e = V.element()
    ep = Vp.element()
    tdim = e.topological_dimension()
    assert tdim == 2, "I need topological dimension 2 (was %d)" % tdim
    coords = V.tabulate_dof_coordinates().reshape((-1, tdim))

    Dh = PETSc.Mat()
    Dh.create(PETSc.COMM_WORLD)
    Dh.setSizes([Vp.dim(), V.dim()])
    Dh.setType("aij")
    Dh.setUp()
    # In order not to assume the following it would be enough
    # to create the array inside the loop...
    warning("Assuming all cells have the same number of dofs")

    ## Preinit element-wise matrix
    M = np.zeros((ep.space_dimension(), e.space_dimension()), dtype=np.float)
    rows = [[0,1], [2,3], [4,5], [6,7], [8,9], [10,11]]
    # Hack to make fancy indexing with lists work:
    # if one of rows[i], cols[j] has shape (2,1), broadcasting will actually
    # compute the outer product of rows[i] and cols[j], instead of a "zip"
    # of sorts.
    rows = list(map(lambda x: np.array(x).reshape((2,1)), rows))
    cols = [0, [1,2], 3, [4,5], 6, [7,8]]
    #cols = list(map(lambda x: np.array(x).reshape((-1,1)), cols))
    M[rows[0],cols[1]] = np.eye(2)
    M[rows[1],cols[3]] = np.eye(2)
    M[rows[2],cols[5]] = np.eye(2)
    M[rows[3],cols[1]] = -0.5*np.eye(2)
    M[rows[4],cols[3]] = -0.5*np.eye(2)
    M[rows[5],cols[5]] = -0.5*np.eye(2)

    # Allocate temporary arrays
    numpt = 3
    tt = np.empty((numpt, tdim))
    TT = np.empty((numpt, tdim, tdim))
    ss = np.empty(numpt)

    # TODO: check this for parallel operation...
    istart, iend = Dh.getOwnershipRange()
    print ("This process owns the range %d - %d" % (istart, iend))

    for cell_id in range(V.mesh().num_cells()):
        ## Compute tangents and sub-submatrices
        c = Cell(V.mesh(), cell_id)
        cc = c.get_vertex_coordinates().reshape((numpt, 2))
        for i in range(numpt):
            tt[i] = (cc[(i+1)%numpt] - cc[i%numpt])
            ss[i] = np.linalg.norm(tt[i], ord=2)
            tt[i] /= ss[i]
            TT[i] = -3./4 * np.outer(tt[i], tt[i])
            tt[i] *= -3./(2*ss[i])
        M[rows[3], cols[2]] = tt[0].copy().reshape((2,1))
        M[rows[3], cols[3]] = TT[0].copy()
        M[rows[3], cols[4]] = -tt[0].copy().reshape((2,1))
        M[rows[3], cols[5]] = TT[0].copy()

        M[rows[4], cols[0]] = tt[1].copy().reshape((2,1))
        M[rows[4], cols[1]] = TT[1].copy()
        M[rows[4], cols[4]] = -tt[1].copy().reshape((2,1))
        M[rows[4], cols[5]] = TT[1].copy()

        M[rows[5], cols[0]] = tt[2].copy().reshape((2,1))
        M[rows[5], cols[1]] = TT[2].copy()
        M[rows[5], cols[2]] = -tt[2].copy().reshape((2,1))
        M[rows[5], cols[3]] = TT[2].copy()

        # Massage dofs into the ordering we require with
        # both coordinates for each vector-valued dof consecutive.
        dest_rows = np.array(list(chain.from_iterable(zip(dmp1.cell_dofs(cell_id),
                                                     dmp2.cell_dofs(cell_id)))),
                             dtype=np.intc)
        dest_rows = dest_rows[(dest_rows>=istart) & (dest_rows < iend)]
        # dofs in V are already in the proper ordering (point eval, gradient eval, ...)
        dest_columns = dm.cell_dofs(cell_id)
        dof_coords = coords[dest_columns,:]

        # NOTE: the copy() is necessary!
        #print("Setting:\n\trows=%s\n\tcols=%s\nwith:\n%s" % (dest_rows, dest_columns, M))
        Dh.setValues(dest_rows, dest_columns, M)
        assert len(dest_rows)*len(dest_columns) ==  M.size, "duh"
    Dh.assemble()
    return Dh

In [ ]:
def compute_dkt_gradient(V, Vp, u):
    """ Returns the discrete gradient of u.

    Arguments
    ---------
        V: Source FunctionSpace of type DKT. (redundant...)
        Vp: Target VectorFunctionSpace of type CG2.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """
    Dh = dkt_gradient_operator(V, Vp)
    du = Function(Vp)
    petsc_u = as_backend_type(u.vector()).vec()
    petsc_du = as_backend_type(du.vector()).vec()
    
    Dh.mult(petsc_u, petsc_du)
    du.vector().apply('insert')

    return du

In [ ]:
from dolfin import *
import nbimporter

W = FunctionSpace(UnitSquareMesh(1,1), 'DKT', 3)
T = VectorFunctionSpace(W.mesh(), 'Lagrange', 2, dim=2)
u = TrialFunction(W)
v = TestFunction(W)

Dh = dkt_gradient_operator(W, T)